In [2]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


In [5]:
real_dir = "D:\Projects\deepfake detection project\deepfake detection\Real"
fake_dir ="D:\Projects\deepfake detection project\deepfake detection\Fake"


In [6]:
def extract_frames(video_dir, image_dir):
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    for video in os.listdir(video_dir):
        cap = cv2.VideoCapture(os.path.join(video_dir, video))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_index = np.random.randint(0, frame_count)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        success, frame = cap.read()
        if success:
            frame = cv2.resize(frame, (224, 224))
            video_name = os.path.splitext(video)[0]
            image_name = video_name + "_" + str(frame_index) + ".jpg"
            cv2.imwrite(os.path.join(image_dir, image_name), frame)
        cap.release()

extract_frames(real_dir, "D:\deepfake detection project\deepfake detection/real_ph")
extract_frames(fake_dir, "D:\deepfake detection project\deepfake detection/fake_ph")


In [18]:
def load_data(image_dirs, labels):
    images = []
    image_labels = []
    for image_dir, label in zip(image_dirs, labels):
        for image in os.listdir(image_dir):
            img = cv2.imread(os.path.join(image_dir, image))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.0 
            images.append(img)
            image_labels.append(label)
            break
    images = np.array(images)
    image_labels = np.array(image_labels)
    return images, image_labels


In [19]:
X, y = load_data(["D:\deepfake detection project\deepfake detection/real_ph","D:\deepfake detection project\deepfake detection/fake_ph" ], [0, 1])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [33]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    return model
model = create_model()


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss", mode="min")
model.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.1, callbacks=[checkpoint])
model.evaluate(X_test, y_test)
model.load_weights("best_model.h5")
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


In [ ]:
model.save("best_modelv1.h5")